# O(1) Memory Statistics

Guido Kraemer (Leipzig University)

# Statistics

  Activating project at `~/Documents/classes/spatio-temporal_data/classes/03_o1_memory_stats`

## sum

Let’s calculate the sum of `n` random numbers. Which one of these
functions consumes less memory as `n` increases?

In [2]:
function rand_sum1(n)
    x = rand(n)
    sum(x)
end

function rand_sum2(n)
    s = 0.0
    for i in 1:n
        s += rand()
    end
    s
end

## memory

In [3]:
for i in 1:4
    n = 10^i
    println("n = $n")
    print("  rand_sum1: ")
    @time rand_sum1(n);
    print("  rand_sum2: ")
    @time rand_sum2(n);
end

n = 10
  rand_sum1:   0.000003 seconds (1 allocation: 144 bytes)
  rand_sum2:   0.000002 seconds
n = 100
  rand_sum1:   0.000004 seconds (1 allocation: 896 bytes)
  rand_sum2:   0.000001 seconds
n = 1000
  rand_sum1:   0.000004 seconds (1 allocation: 7.938 KiB)
  rand_sum2:   0.000002 seconds
n = 10000
  rand_sum1:   0.000012 seconds (2 allocations: 78.172 KiB)
  rand_sum2:   0.000012 seconds

## large `n`

-   Memory consumption of `rand_sum1` increases linearly with `n`,
    i.e. they are “$\mathcal{O}(n)$ in memory”.
-   If memory consumption is constant, it is called “$\mathcal{O}(1)$ in
    memory”.
-   If `n` is too large, the won’t be enough RAM.
-   There are algorithms that are “worse”, e.g. $\mathcal{O}(n^2)$ or
    even worse.
-   The same annotation can be used for compute time, then it is called
    “$\mathcal{O}(n)$ in time”.

## time complexity

What is the time complexity of `rand_sum2`?

In [4]:
function rand_sum2(n)
    s = 0.0
    for i in 1:n
        s += rand()
    end
    s
end

## $O(n^2)$ example

In [5]:
function rand_sum3(n)
    s = 0.0
    for i in 1:n
        for j in 1:n
            s += rand()
        end
    end
    s
end

-   What is the memory complexity here?
-   What would be a version with with $\mathcal{O}(n^2)$ memory
    complexity?

# $\mathcal{O}(1)$ memory statistics

## How is it possible?

$$s = \sum_{i = 1}^n x_i$$

In [6]:
function my_sum(x::Vector{T}) where T
    s = T(0.0)
    for xi in x
        s += xi
    end
    s
end

## WeightedOnlineStats.jl

In [7]:
using WeightedOnlineStats
o1 = WeightedSum()
for i in 1:10
    xi = rand()
    o1 = fit!(o1, xi, 1)
end
sum(o1)

4.4057707582226175

## More statistics

Sum $$\sum_{i=1}^n x_i$$

Mean $$\frac 1 n \sum_{i=1}^n x_i$$

Variance $$\frac{1}{n} \sum_{i=1}^n (\bar{x} - x_i)^2$$

## Julia

The following Statistics are implemented in `WeightedOnlineStats.jl`:

    WeightedSum
    WeightedMean
    WeightedVariance
    WeightedCovMatrix
    WeightedHist
    WeightedAdaptiveHist

There are more in `OnlineStats.jl`

-   [OnlineStats.jl](https://joshday.github.io/OnlineStats.jl/stable/)
-   [WeightedOnlineStats.jl](https://weightedonlinestats.guido-kraemer.com/stable/)

## Interface

In [8]:
T = Float64
o1 = WeightedMean(T)
o2 = WeightedMean(T)
value1 = rand()
weight1 = rand()
fit!(o1, value1, weight1)
value2 = rand()
weight2 = rand()
fit!(o2, value2, weight2)
o = merge(o1, o2)
mean(o)

0.6982647156029932

## Interface II

We can use vectors but this is the same as a `for`-loop

In [9]:
x = rand(10)
w = rand(10)
fit!(o, x, w)
mean(o)

0.5459058647728202

## Warning

In [10]:
my_mean(x) = my_sum(x) / length(x)
x = Float32.((1_000_000:-1:1) .^ 3)
@show my_mean(x)
@show my_mean(reverse(x))
@show mean(x)
o1 = WeightedMean(Float32)
o2 = WeightedMean(Float64)
for i in 1:length(x)
    fit!(o1, x[i], 1)
    fit!(o2, x[i], 1)
end
@show mean(o1)
@show mean(o2);

my_mean(x) = 2.4973105f17
my_mean(reverse(x)) = 2.4998555f17
mean(x) = 2.500005f17
mean(o1) = 2.5005301f17
mean(o2) = 2.5000050001806928e17

. . .

-   You are responsible for correct calculations!
-   `WeightedOnlineStats.jl` uses algorithms to partially avoid these
    issues.
-   `WeightedOnlineStats.jl` can use higher precision numbers (e.g.
    `Quadmath.Float128`) and `merge!` to avoid these issues

## Catastrophic cancellation

Adding two numbers with 5 digits of precision

      1.2345
    + 0.0012345
      ---------
      1.2356

We are loosing decimals and these losses can add up over millions of
numbers.

# Data Cubes

## Apply it to the data cube

In [11]:
using YAXArrays
using Zarr
c2_path = "/home/gkraemer/data/DataCube/v3.0.2/esdc-8d-0.25deg-1x720x1440-3.0.2.zarr"
c2_zarr = Zarr.zopen(c2_path)
c2_dataset = YAXArrays.open_dataset(c2_zarr)
c2_full = YAXArrays.Cube(c2_dataset)
c2 = c2_full[variable = "air_temperature_2m"]

YAXArray with the following dimensions
lon                 Axis with 1440 Elements from -179.875 to 179.875
lat                 Axis with 720 Elements from -89.875 to 89.875
time                Axis with 1978 Elements from 1979-01-05T00:00:00 to 2021-12-31T00:00:00
units: mm d^-1
Total size: 7.64 GB

## The naive way

In [12]:
using ProgressMeter
function get_mean1(c)
    o = WeightedMean(Float64)
    w = cosd.(getAxis("lat", c).values)
    @showprogress for t in 1:size(c, 3)
        for lat in 1:size(c, 2)
            for lon in 1:size(c, 1)
                fit!(o, c[lon, lat, t], w[lat])
            end
        end
    end
    mean(o)
end
# get_mean1(c2) # dont't run!

get_mean1 (generic function with 1 method)

Don’t run this, it will never finish! Why?

## Efficient reading of data

In [13]:
function get_mean2(c)
    o = WeightedMean(Float64)
    w = cosd.(getAxis("lat", c).values)
    @showprogress for t in 1:size(c, 3)
        ct = c[:, :, t] # respect chunking
        for lat in 1:size(c2, 2)
            wlat = w[lat]
            for lon in 1:size(c, 1)
                fit!(o, ct[lon, lat], wlat)
            end
        end
    end
    mean(o)
end
# get_mean2(c2) # < 4 min

get_mean2 (generic function with 1 method)

## Parallel processing

Modern CPUs have more than one CPU!

In [14]:
@show Threads.nthreads()
Threads.@threads for i in 1:10
    println("Hello from thread $(Threads.threadid())")
end

Threads.nthreads() = 8
Hello from thread 1
Hello from thread 7
Hello from thread 4
Hello from thread 5
Hello from thread 4
Hello from thread 2
Hello from thread 8
Hello from thread 3
Hello from thread 3
Hello from thread 6

Julia can use threading. An alternative is `Distributed` to use separate
processes. - Threads share memory, processes don’t

## Adapt our algorithm

In [15]:
function get_mean3(c)
    ntime = length(getAxis("time", c))
    w = cosd.(getAxis("lat", c))
    m = [WeightedMean(Float64) for _ in 1:ntime]
    p = Progress(ntime)
    Threads.@threads for t in 1:size(c, 3)
        ct = c[:, :, t] # respect chunking
        for lat in 1:size(ct, 2)
            wlat = w[lat]
            for lon in 1:size(ct, 1)
                fit!(m[t], ct[lon, lat], wlat)
            end
        end
        next!(p)
    end
    m0 = reduce(merge!, m) # pairwise merging reduces error!
    mean(m0)
end
# get_mean3(c2) # ~ 1 min

-   Why is `@threads` in the outermost loop?
-   Is this calculation CPU or Bandwith starved?

# Exercises

## Exercises

-   try get_mean with different accuracies (`Float32` and `Float16`,
    `Quadmath.Float128`), Does the result change?
-   there is `mean(x::WeightedVariance)`, does this differ from the
    result of `mean(x::WeightedMean)`
-   Create a cube with a single variable
    -   z-transform each timeseries using `mapCube`

## Exercises 2

-   Create a cube with 6 variables

-   create a cube with the same six variables and an axis with values
    “mean” and “std”

    -   Create a vector of WeightedOnlineStats

        ``` julia
        means = [WeightedMean(Float64) for _ in 1:6]
        stds = [WeightedVariance(Float64) for _ in 1:6]
        ```

    -   You can create a cube from an array as follows:

        ``` julia
        axlist = [CategoricalAxis("statistic", ["mean", "std"]),
                  getAxis("Variable", c2_full)]
        data = rand(length.(axlist)...)
        YAXArray(axlist, data)
        ```

    -   Where is the best place to implement parallelization?

-   use `mapCube` to z-transform the variables

    ``` julia
    mapCube((c1, c2), indims = (InDims(...), InDims(...)), outdims = ...) do xout, xin1, xin2
      ...
    end
    ```